In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import ssl

In [35]:
batch_size = 256# 批大小
epochs = 20 # 训练轮数
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

In [36]:
ssl._create_default_https_context = ssl._create_unverified_context# 忽略ssl证书验证

In [37]:

train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=True, 
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True)
       
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True)

In [38]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)#1输入通道，10输出通道，5x5卷积核
        self.conv2 = nn.Conv2d(10, 20, kernel_size=3)#10输入通道，20输出通道，5x5卷积核
        self.fc1 = nn.Linear(20 * 10 * 10, 500)#20*10*10输入，500输出 全连接层
        self.fc2 = nn.Linear(500, 10)#500输入，10输出 全连接层
        
    def forward(self, x):
        input_size = x.size(0)#输入大小 #batch_size x 1 x 28 x 28 张量形式
        x = self.conv1(x)#卷积层1 输入：batch_size x 1 x 28 x 28 输出：batch_size x 10 x 24 x 24
        x = F.relu(x)#激活函数 保持输入大小不变
        x = F.max_pool2d(x, 2, 2)#池化层 输入：batch_size x 10 x 24 x 24 输出：batch_size x 10 x 12 x 12 降采样2倍
        x = self.conv2(x)#卷积层2 输入：batch_size x 10 x 12 x 12 输出：batch_size x 20 x 10 x 10
        x = F.relu(x)#激活函数 保持输入大小不变
        x = x.view(input_size, -1)#展平 输入：batch_size x 20 x 10 x 10 输出：batch_size x 320 自动计算维度
        x = self.fc1(x) #全连接层1 输入：batch_size x 320 输出：batch_size x 50
        x = F.relu(x)
        x = self.fc2(x)#全连接层2 输入：batch_size x 50 输出：batch_size x 10
        output = F.log_softmax(x,dim=1)#计算分类后每个数字的概率值
        return output

In [39]:
#定义优化器
model = Net().to(device)
optimizer = optim.Adam(model.parameters())#使用Adam优化器


In [40]:
#定义训练方法
def train_model(model, device,train_loader, optimizer, epoch):#训练模型
    model.train()#训练模式
    for batch_index, (data, target) in enumerate(train_loader):#遍历训练集
        data, target = data.to(device), target.to(device)#数据转移到GPU
        optimizer.zero_grad()#梯度初始化
        #预测
        output = model(data)#模型输出
        #计算损失
        loss = F.cross_entropy(output, target)#交叉熵损失 适用于多分类问题
        #找到概率值最大的类别
        pred = output.max(1, keepdim=True)
        #反向传播
        loss.backward()
        optimizer.step()#更新参数
        if batch_index % 3000 == 0:#每3000个batch打印一次信息
            print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch,loss.item())) #打印损失值

In [41]:
#定义测试方法
def test_model(model, device, test_loader):#测试模型
    model.eval()#测试模式
    #正确率计数器
    correct = 0
    #损失值计数器 
    test_loss = 0
    with torch.no_grad():#禁用梯度计算 测试与训练分开
        for data, target in test_loader:#遍历测试集
            data, target = data.to(device), target.to(device)#数据转移到GPU
            #测试数据
            output = model(data)
            #计算损失
            test_loss += F.cross_entropy(output, target).item()#累加损失值
            #找到概率值最大的类别
            pred = output.max(1, keepdim=True)[1]
            #累计正确率
            correct += pred.eq(target.view_as(pred)).sum().item()
            #计算平均损失值
    test_loss /= len(test_loader.dataset)
    print("Test -- Average loss: {:.4f}, Accuracy: {:.3f}\n".format(test_loss, 100.0 * correct / len(test_loader.dataset)))

In [42]:
#调用方法训练模型
for epoch in range(1, epochs + 1):
    train_model(model, device, train_loader, optimizer, epoch)
    test_model(model, device, test_loader)
    

Train Epoch: 1 	Loss: 2.327436
Test -- Average loss: 0.0003, Accuracy: 98.010

Train Epoch: 2 	Loss: 0.063697
Test -- Average loss: 0.0002, Accuracy: 98.410

Train Epoch: 3 	Loss: 0.036108
Test -- Average loss: 0.0002, Accuracy: 98.270

Train Epoch: 4 	Loss: 0.045501
Test -- Average loss: 0.0001, Accuracy: 98.810

Train Epoch: 5 	Loss: 0.019364
Test -- Average loss: 0.0002, Accuracy: 98.710

Train Epoch: 6 	Loss: 0.055409
Test -- Average loss: 0.0002, Accuracy: 98.790

Train Epoch: 7 	Loss: 0.014359
Test -- Average loss: 0.0001, Accuracy: 99.150

Train Epoch: 8 	Loss: 0.001435
Test -- Average loss: 0.0001, Accuracy: 99.000

Train Epoch: 9 	Loss: 0.007028
Test -- Average loss: 0.0001, Accuracy: 98.920

Train Epoch: 10 	Loss: 0.004547
Test -- Average loss: 0.0001, Accuracy: 99.090

Train Epoch: 11 	Loss: 0.002823
Test -- Average loss: 0.0002, Accuracy: 98.920

Train Epoch: 12 	Loss: 0.000838
Test -- Average loss: 0.0001, Accuracy: 99.010

Train Epoch: 13 	Loss: 0.000370
Test -- Average l